# Introduction

This is the pre-processing notebook for the European Commission and White House datasets. This notebook aims to standardize and prepare the data for analysis. Here are the main steps:

- Casting Date Format: Standardizing date format for consistency.
- Filtering Data: Including records from 2021-2023 for three-year panel data (so that we have 2021 for pre-war comparison as well).
- Updating Category Names: Unifying category labels across datasets.
- Excluding Non-English Texts (EC Dataset): Removing non-English texts for data integrity.
- Lemmatizing Data: Reducing words to base form for analysis.
- Adding Binary Column for Mentions of Ukraine: Identifying mentions of Ukraine for analysis.

These steps ensure data consistency and usability for insightful analysis.

# Set up the environment

## Import Libraries

In [6]:
import pandas as pd
from langdetect import detect

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords

#!python -m spacy download en_core_web_sm
import spacy
sp = spacy.load('en_core_web_sm')

import re


from tqdm import tqdm
tqdm.pandas()


#getting a library of stopwords and defining a lemmatizer
porter=SnowballStemmer("english")
lmtzr = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/viktoriia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Additional functions

In [4]:
def strip(word):
    mod_string = re.sub(r'\W+', '', word)
    return mod_string


# The following leaves in place two or more capital letters in a row
# Will be ignored when using standard stemming
def abbr_or_lower(word):
    if re.match('([A-Z]+[a-z]*){2,}', word):
        return word
    else:
        return word.lower()

# Modular pipeline for stemming, lemmatizing and lowercasing
# Note this is NOT lemmatizing using grammar pos
    
def tokenize(text, modulation):
    if modulation<2:
        tokens = re.split(r'\W+', text)
        stems = []
        # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
        for token in tokens:
            lowers=abbr_or_lower(token)
            if lowers not in stop_words:
                if re.search('[a-zA-Z]', lowers):
                    if modulation==0:
                        stems.append(lowers)
                    if modulation==1:
                        stems.append(porter.stem(lowers))
    else:
        sp_text=sp(text)
        stems = []
        lemmatized_text=[]
        for word in sp_text:
            lemmatized_text.append(word.lemma_)
        stems = [abbr_or_lower(strip(w)) for w in lemmatized_text if (abbr_or_lower(strip(w))) and (abbr_or_lower(strip(w)) not in stop_words)]
    return " ".join(stems)

# Load the data

In [7]:
# Load the European Commission data
ec_data = pd.read_csv('data/theeuropeancommission.csv')
ec_data.head()

,Title,Date,Category,Text
0,Joint press statement by the President of the ...,8 February 2024,STATEMENT,"On 8 February 2024 Her Excellency, President o..."
1,Press statement by President von der Leyen fol...,8 February 2024,STATEMENT,"President Ghazouani,\nThank you for your warm ..."
2,Opening remarks by President von der Leyen at ...,1 February 2024,STATEMENT,Today is a very special day. The European Coun...
3,ICJ: Joint Statement by the High Representativ...,26 January 2024,STATEMENT,We take note of today's order of the Internati...
4,Statement by the Commission and High Represent...,26 January 2024,STATEMENT,We are extremely concerned by allegations of U...


In [48]:
# Load the White House data
wh_data = pd.read_csv('data/thewhitehouse.csv')
wh_data.head()

,Title,Date,Category,Location,Text
0,Remarks by President Biden and Vice President ...,2024-02-03T22:00:00-05:00,Speeches and Remarks,Biden for President Campaign Headquarters; Wil...,"THE VICE PRESIDENT: Hello, Delaware! (Applau..."
1,Remarks by Vice President Harris at a Campaign...,2024-02-02T23:33:00-05:00,Speeches and Remarks,"South Carolina State University; Orangeburg, S...",THE VICE PRESIDENT: All right. Can we hear i...
2,Remarks by President Biden at a Political Even...,2024-02-01T20:24:19-05:00,Speeches and Remarks,"Region 1 Union Hall; Warren, Michigan","4:41 P.M. EST\n \nTHE PRESIDENT: Well, thank ..."
3,Remarks by President Biden at the National Pra...,2024-02-01T14:13:03-05:00,Speeches and Remarks,"U.S. Capitol; Washington, D.C.","9:04 A.M. EST\nTHE PRESIDENT: Frank, thank yo..."
4,Remarks by President Biden at a Campaign Recep...,2024-01-31T00:04:32-05:00,Speeches and Remarks,"Private Residence; Miami, Florida","6:27 P.M. EST\n\nTHE PRESIDENT: Well, Chris, t..."


# Clean the data

## The European Commission data

#### Clean the Date column:

In [10]:
# Convert 'Date' column to datetime
ec_data['Date'] = pd.to_datetime(ec_data['Date'], format='%d %B %Y')

# Filter out dates between 2021 and 2023 so we have 3 years panel data
ec_data = ec_data[(ec_data['Date'] >= '2021-01-01') & (ec_data['Date'] <= '2023-12-31')]

# Sort the dataset in the ascending order of date
ec_data = ec_data.sort_values(by = 'Date', ignore_index=True)

ec_data.head()

,Title,Date,Category,Text
0,European Commission authorises second safe and...,2021-01-06,PRESS RELEASE,"Today, the European Commission has granted a c..."
1,Statement by President von der Leyen on the re...,2021-01-07,STATEMENT,What happened in Washington yesterday is as ...
2,Commission proposes to purchase up to 300 mill...,2021-01-08,PRESS RELEASE,The European Commission today proposed to the ...
3,Speech by President von der Leyen at the One P...,2021-01-11,SPEECH,"Merci Monsieur le Président.\nCher Emmanuel,\n..."
4,Coronavirus: Commission concludes exploratory ...,2021-01-12,PRESS RELEASE,"Today, the European Commission concluded explo..."


#### Ensure that the category names are aligned between the two datasets:

In [8]:
ec_data.Category.unique()

array(['STATEMENT', 'SPEECH', 'PRESS RELEASE'], dtype=object)

In [11]:
# Define the replacement dictionary
replacement_dict = {
    'PRESS RELEASE': 'Press Release',
    'STATEMENT': 'Statement',
    'SPEECH': 'Speech'
}

# Replace values in the 'column_name' column using the replace() method
ec_data['Category'] = ec_data['Category'].replace(replacement_dict)

#### Check for Null values and duplicates:

In [12]:
# Check for null values
ec_data.isna().sum()

Title       0
Date        0
Category    0
Text        0
dtype: int64

In [13]:
# Check for duplicates
ec_data.duplicated().any()

False

There are no Null values and duplicates in this dataset.

#### Exclude texts that are not in English from the dataset:

Upon observation, we identified texts within the dataset that are not in English. To streamline further analysis and ensure consistency, we've opted to remove these non-English texts from the dataset.

In [14]:
# Drop articles that are not in English

non_en_count = 0
for index, row in ec_data.iterrows():
    lang = detect(row['Text'])
    if lang != 'en':  # 'en' represents English
        non_en_count +=1
        ec_data = ec_data.drop(index=index)
print(f'Number of non-English articles: {non_en_count}')

Number of non-English articles: 21


#### Pre-process titles and texts:

In [16]:
# 0 - lowercase, 1 - stemming, 2 - lemmatizing 
mod=2

# Pre-process texts
text_preproc = (
    ec_data.Text
    .astype(str)
    .progress_apply(lambda row: tokenize(row, mod))
)

ec_data["text_preproc"]=text_preproc

print("Done with text!")

# Pre-process titles
tit_preproc = (
    ec_data.Title
    .astype(str)
    .progress_apply(lambda row: tokenize(row, mod))
)

ec_data["titles_preproc"]=tit_preproc

print("Done with titles!")
ec_data.info()

100%|███████████████████████████████████████| 1078/1078 [02:57<00:00,  6.07it/s]


Done with text!


100%|██████████████████████████████████████| 1078/1078 [00:04<00:00, 225.30it/s]

Done with titles!
<class 'pandas.core.frame.DataFrame'>
Index: 1078 entries, 0 to 1098
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Title           1078 non-null   object        
 1   Date            1078 non-null   datetime64[ns]
 2   Category        1078 non-null   object        
 3   Text            1078 non-null   object        
 4   text_preproc    1078 non-null   object        
 5   titles_preproc  1078 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 59.0+ KB


#### Add a binary column to detect mentions of Ukraine:

We're enriching our dataset by introducing a binary column to indicate the presence of mentions related to Ukraine. In this new column, a value of 1 will signify that the text contains at least one mention of "Ukraine" or "Ukrainian", and 0 otherwise.

In [17]:
# Create a new column with 1 if 'Text' contains "ukraine"/"ukrainian", 0 otherwise
ec_data['Ukraine'] = ec_data['Text'].str.lower().str.contains('ukrain').astype(int)

In [18]:
# Shape of the final df
ec_data.shape

(1078, 7)

#### Save the pre-processed data:

In [64]:
ec_data.to_csv('data/theeuropeancommission_preprocessed.csv', index=False)

## The White House data

#### Clean the Date column:

In [49]:
# Remove everything after 'T' character
wh_data['Date'] = wh_data['Date'].str.split('T').str[0]

# Convert 'Date' column to datetime
wh_data['Date'] = pd.to_datetime(wh_data['Date'])

# Filter out dates between 2021 and 2023
wh_data = wh_data[(wh_data['Date'] >= '2021-01-01') & (wh_data['Date'] <= '2023-12-31')]

# Sort the dataset in the ascending order of date
wh_data = wh_data.sort_values(by = 'Date', ignore_index=True)

wh_data.head()

,Title,Date,Category,Location,Text
0,Fact Sheet: President-elect Biden Outlines COV...,2021-01-15,Statements and Releases,NaN,Effectively and equitably vaccinating the U.S....
1,Fact Sheet: List of Agency Actions for Review,2021-01-20,Statements and Releases,NaN,"Actions Address the COVID-19 Pandemic, Provide..."
2,Fact Sheet: President Biden Sends Immigration ...,2021-01-20,Statements and Releases,NaN,The U.S. Citizenship Act of 2021 establishes a...
3,"A Letter to Dr. Eric S. Lander, the President’...",2021-01-20,Statements and Releases,NaN,"On January 15, 2021, then-President-Elect Bide..."
4,Paris Climate Agreement,2021-01-20,Statements and Releases,NaN,ACCEPTANCE ON BEHALF OF THE UNITED STATES OF A...


#### Ensure that the category names are aligned between the two datasets:

In [50]:
wh_data.Category.unique()

array(['Statements and Releases', 'Speeches and Remarks',
       'Press Briefings'], dtype=object)

In [51]:
# Define the replacement dictionary
replacement_dict = {
    'Press Briefings': 'Press Release',
    'Statements and Releases': 'Statement',
    'Speeches and Remarks': 'Speech'
}

# Replace values in the 'column_name' column using the replace() method
wh_data['Category'] = wh_data['Category'].replace(replacement_dict)

#### Drop the location column as it is not relevant for our analysis:

In [52]:
wh_data = wh_data.drop('Location', axis=1)

#### Check for Null values and duplicates:

In [53]:
# Check for null values
wh_data.isna().sum()

Title       0
Date        0
Category    0
Text        1
dtype: int64

We've identified one record with a null value in the Text column. Additionally, during our data exploration, we found another record containing only '###' in the Text column. These entries will be removed from the dataset.

In [54]:
# Remove Text = Null and Text = ### rows from the data set
wh_data = wh_data[(wh_data['Text'].notnull()) & (wh_data['Text'] != '###')]

In [55]:
# Check for duplicates
wh_data.duplicated().any()

True

As we can see, there some duplicate rows in this dataset. Let's check them:

In [56]:
wh_data[wh_data.duplicated(keep=False)]

,Title,Date,Category,Text
405,Remarks By Vice President Harris In A Virtual ...,2021-03-25,Speech,"6:02 P.M. EDT\n\nTHE VICE PRESIDENT: Hi, Rabb..."
408,Remarks By Vice President Harris In A Virtual ...,2021-03-25,Speech,"6:02 P.M. EDT\n\nTHE VICE PRESIDENT: Hi, Rabb..."
4521,Background Press Call on President Biden’s Exe...,2022-09-15,Press Release,"MODERATOR: Great, thank you. Hey, everyone. ..."
4538,Background Press Call on President Biden’s Exe...,2022-09-15,Press Release,"MODERATOR: Great, thank you. Hey, everyone. ..."
6338,Remarks by President Biden and President Volod...,2023-05-21,Speech,"2:46 P.M. JST\n\nPRESIDENT BIDEN: Well, Mr. P..."
6339,Remarks by President Biden and President Volod...,2023-05-21,Speech,"2:46 P.M. JST\n\nPRESIDENT BIDEN: Well, Mr. P..."


In [57]:
# Drop duplicates from the dataset
wh_data = wh_data.drop_duplicates(keep='first')

#### Pre-process titles and texts:

In [58]:
# 0 - lowercase, 1 - stemming, 2 - lemmatizing 
mod=2

# Pre-process texts
text_preproc = (
    wh_data.Text
    .astype(str)
    .progress_apply(lambda row: tokenize(row, mod))
)

wh_data["text_preproc"]=text_preproc

print("Done with text!")

# Pre-process titles
tit_preproc = (
    wh_data.Title
    .astype(str)
    .progress_apply(lambda row: tokenize(row, mod))
)

wh_data["titles_preproc"]=tit_preproc

print("Done with titles!")
wh_data.info()

100%|███████████████████████████████████████| 7846/7846 [44:06<00:00,  2.96it/s]


Done with text!


100%|██████████████████████████████████████| 7846/7846 [00:34<00:00, 229.93it/s]

Done with titles!
<class 'pandas.core.frame.DataFrame'>
Index: 7846 entries, 0 to 7850
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Title           7846 non-null   object        
 1   Date            7846 non-null   datetime64[ns]
 2   Category        7846 non-null   object        
 3   Text            7846 non-null   object        
 4   text_preproc    7846 non-null   object        
 5   titles_preproc  7846 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 429.1+ KB


#### Add a binary column to detect mentions of Ukraine:

In [59]:
# Create a new column with 1 if 'Text' contains "ukraine"/"ukrainian", 0 otherwise
wh_data['Ukraine'] = wh_data['Text'].str.lower().str.contains('ukrain').astype(int)

In [60]:
# Shape of the final df
wh_data.shape

(7846, 7)

#### Save the pre-processed data:

In [63]:
wh_data.to_csv('data/thewhitehouse_preprocessed.csv', index=False)

In [66]:
# File is too big to be pushed on git, so we splitted in into two
split_index = len(wh_data) // 2  # Calculate the index to split the DataFrame
wh_data_part1 = wh_data.iloc[:split_index]  # Select first half of rows
wh_data_part2 = wh_data.iloc[split_index:]

In [71]:
wh_data_part1.to_csv('data/thewhitehouse_preprocessed_p1.csv', index=False)
wh_data_part2.to_csv('data/thewhitehouse_preprocessed_p2.csv', index=False)

# Conclusion

In this pre-processing notebook, we successfully cleaned and prepared the European Commission and White House datasets for analysis. By standardizing date formats, filtering data, unifying category names, excluding non-English texts (for the European Commission dataset), lemmatizing text data, and adding a binary column for mentions of Ukraine, we have ensured data consistency and usability.

These pre-processing steps lay a solid foundation for insightful analysis and informed decision-making. With the data now refined and ready for exploration, we can delve into further analyses to uncover meaningful insights and trends. Through rigorous pre-processing, we have maximized the potential of our datasets and set the stage for comprehensive analysis.